<h1 id="toctitle">Performance and benchmarking</h1>
<ul id="toc"/>

---

Distinct but related concepts:

- Measuring
    - Benchmarking (how long does something take)
        - CPU
        - Memory (less so)
    - Profiling (which bits are slow)
        - CPU
        - ~~Memory~~ (not today)
- Optimizing

As with most programming jobs, a range of tools from simple to complex.

## Benchmarking

In approximate order of usefulness....

### Unix time
How long does our program take to run? On Linux/Mac we can do 

```
time somecommand
```

In iPython, prefix shell commands with `!`


In [1]:
!date

уто, 20. феб 2018.  17:04:28 CET



Given output that looks like this:

```
real	0m0.490s 
user	0m0.457s 
sys	    0m0.032s 
```

- real is the wallclock time (affected by busy systems and other programs)
- user is the time spent executing our code
- sys is the time spent waiting for system calls (file IO, memory, network)

user+sys is probably the most useful. 

### Manual timing

Just measure the current time at the start of code, then again at end, and get the difference. 

`time.time()` gives us current UNIX epoch (number of seconds since midnight January 1st 1970 (don't ask.))

In [2]:
import time 
time.time()

1519142671.8718295

On most systems this has very high resolution. 

In [3]:
import time 
start = time.time() 

# print the sum of the first million cube numbers
x = 0 
for i in range(1000000): 
    x = x + i ** 3 
print(x) 
 
end = time.time() 
print(end - start) 

249999500000250000000000
0.4049837589263916


This is arguably better than using the `time` command line as it doesn't include Python start up time, etc. However, still affected by other processes.

### `timeit` module

Python has a built in module for doing timing. From the command line:

In [4]:
!python -m timeit "4 ** 10"

100000000 loops, best of 3: 0.0156 usec per loop


Nice features:
- automatically runs the code many times to get an accurate measurement
- runs the whole thing three times and reports the best (accounts for other processes)
- gives the answer in easy to read units:

In [5]:
!python -m timeit "12345 in range(1000000)" 

100 loops, best of 3: 17.8 msec per loop


In the above code, do we spend more time constructing the range list or checking if the number is in it? Let's try just constructing the range:

In [6]:
!python -m timeit "range(1000000)" 

100 loops, best of 3: 17.7 msec per loop


Yep, takes loads of time to construct the list. Seperate that bit out with a setup (`-s`) command:

In [7]:
!python -m timeit -s "r=range(1000000)" "12345 in r" 

10000 loops, best of 3: 106 usec per loop


In iPython, we have magic convenience functions:

In [8]:
%timeit 4 ** 10

14.3 ns ± 0.226 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [9]:
r=range(1000000)
%timeit 12345 in r

136 ns ± 3.99 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [10]:
%%timeit
# print the sum of the first million cube numbers
x = 0 
for i in range(1000000): 
    x = x + i ** 3 
#print(x) 

360 ms ± 5.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


`timeit` is useful for quickly checking which approach is faster. `timeit` case study: which way is faster to calculate AT content - counting a and t, or looking at each base and keeping a tally?

In [11]:
from __future__ import division 

def at_count(dna): 
    return (dna.count('a') + dna.count('t')) / len(dna) 
 
def at_iter(dna): 
    a_count = 0 
    t_count = 0 
    for base in dna: 
        if base == 'a': 
            a_count = a_count + 1 
        elif base == 't': 
            t_count = t_count + 1 
    return (a_count + t_count) / len(dna) 

test_dna = 'atcgatcgatcatgatcggatcgtagctagcatctagtc' 
assert(at_count(test_dna) == at_iter(test_dna)) 

Which is faster?

In [12]:
%timeit at_count(test_dna)

743 ns ± 7.35 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
%timeit at_iter(test_dna)

3.8 µs ± 39.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Hmmm, something odd is going on. Short strings don't give reliable benchmarking results in Python due to optimizations in cPython. Let's try a more realistic input:

In [14]:
import random
def random_dna(length):
    return "".join([random.choice(['A','T','G','C']) for _ in range(length)])

In [15]:
random_dna(20)

'ACAGACGTTCCTCAACCATT'

Now we can compare the two functions:

In [16]:
%timeit at_count(random_dna(10000))
%timeit at_iter(random_dna(10000))

16.3 ms ± 621 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
16.9 ms ± 667 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Looks about equal, but wait: what if most of the time is spent generating the random DNA sequence? This is fairer:

In [17]:
d = random_dna(10000)
%timeit at_count(d)
%timeit at_iter(d)

12.9 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
762 µs ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Summary: 

- getting timing right is hard
- `count()` is faster than iteration (due to fast C code)

### Benchmarking memory

Here's the short story:

`pip install psutil`

then

In [18]:
import psutil, os 
 
process = psutil.Process(os.getpid()) 
mem = process.get_memory_info().rss / 1000 
print("Used this much memory: " + str(mem) + ' kb')

ImportError: No module named 'psutil'

Problem: this is useless in iPython notebooks as it includes everything that's been executed. For simple scripts, it's better. 

In [44]:
!python check_mem.py

Used this much memory: 43216 kb


This lets us investigate time/memory trade offs. We know that checking to see if a number is in a set is faster than checking to see if it's in a list:

In [18]:
l = range(1000000)
s = set(l)
%timeit 12345 in l
%timeit 12345 in s

10000 loops, best of 3: 67.4 µs per loop
10000000 loops, best of 3: 41 ns per loop


but how much longer does it take to create the data structure in the first place?

In [19]:
%timeit list(range(1000000))
%timeit set(range(1000000))

10 loops, best of 3: 25.2 ms per loop
10 loops, best of 3: 48.1 ms per loop


and how much more memory does it take to hold the set?

In [47]:
!python list_mem.py
!python set_mem.py

Used this much memory: 43814 kb
Used this much memory: 68796 kb


Conclusions:
- if we need to create a list once then check membership many times, a set will be faster
- if we need to create many lists, a set might be slower
- a set will use more (x1.5) memory for these ranges 

Of course, everything might be different for non-integers!

## Profiling

Profiling is the process of taking an existing piece of code and identifying which bits are taking the time. 

Scenario: given

- a single long DNA sequence
- a collection of interesting 4-base motifs

we want to identify frequently-occuring (say 50 times) 4-base motifs in the sequence and divide them into ones that are also on the interesting list, and ones that aren't. 



In [21]:
# create a random dna sequence
dna = random_dna(10000)

# create 100 random interesting motifs
motifs = [random_dna(4) for _ in range(100)]

In [22]:
%%timeit
# standard kmer counting code to identify frequent chunks
frequent_chunks = [] 
for start in range(len(dna) - 3): 
    chunk = dna[start:start + 4] 
    if dna.count(chunk) > 50: 
        frequent_chunks.append(chunk) 

# now check each chunk to see if it's in the list of motifs
for chunk in frequent_chunks: 
    if chunk in motifs: 
        print(chunk + " is frequent and interesting") 
    else: 
        print(chunk + " is frequent but not interesting")

GGTG is frequent and interesting
GTAT is frequent but not interesting
TGGG is frequent but not interesting
ACGC is frequent but not interesting
CGCC is frequent but not interesting
TGTC is frequent but not interesting
ATAC is frequent but not interesting
TACG is frequent but not interesting
TGGG is frequent but not interesting
GGTG is frequent and interesting
GTAT is frequent but not interesting
CGCC is frequent but not interesting
TCCT is frequent and interesting
CGGT is frequent but not interesting
GGTG is frequent and interesting
GGAT is frequent and interesting
GATA is frequent but not interesting
TACG is frequent but not interesting
TTCC is frequent and interesting
GGAT is frequent and interesting
GTAT is frequent but not interesting
ATAC is frequent but not interesting
TACG is frequent but not interesting
CGGT is frequent but not interesting
TCCG is frequent but not interesting
GATA is frequent but not interesting
ATAC is frequent but not interesting
TGTC is frequent but not inte

How can we speed this program up? We know that checking to see if an element is in a list is slow, so let's change it to a set:

In [23]:
# create 100 random interesting motifs
motifs = set([random_dna(4) for _ in range(100)])

In [24]:
%%timeit
# standard kmer counting code to identify frequent chunks
frequent_chunks = [] 
for start in range(len(dna) - 3): 
    chunk = dna[start:start + 4] 
    if dna.count(chunk) > 50: 
        frequent_chunks.append(chunk) 

# now check each chunk to see if it's in the list of motifs
for chunk in frequent_chunks: 
    if chunk in motifs: 
        print(chunk + " is frequent and interesting") 
    else: 
        print(chunk + " is frequent but not interesting")
        
print(len(frequent_chunks))

GGTG is frequent but not interesting
GTAT is frequent but not interesting
TGGG is frequent and interesting
ACGC is frequent but not interesting
CGCC is frequent and interesting
TGTC is frequent and interesting
ATAC is frequent and interesting
TACG is frequent but not interesting
TGGG is frequent and interesting
GGTG is frequent but not interesting
GTAT is frequent but not interesting
CGCC is frequent and interesting
TCCT is frequent and interesting
CGGT is frequent but not interesting
GGTG is frequent but not interesting
GGAT is frequent but not interesting
GATA is frequent but not interesting
TACG is frequent but not interesting
TTCC is frequent but not interesting
GGAT is frequent but not interesting
GTAT is frequent but not interesting
ATAC is frequent and interesting
TACG is frequent but not interesting
CGGT is frequent but not interesting
TCCG is frequent but not interesting
GATA is frequent but not interesting
ATAC is frequent and interesting
TGTC is frequent and interesting
GTAT

Why did this not work? Probably because the line

```python
    if chunk in motifs: 
```

doesn't actually get executed that often.

What we need is a way to see which lines are taking up the most time.

### Profiling with cProfile

`cProfile` is a built in module for profiling functions. It's easy to use:

In [57]:
import cProfile 

# we have to turn the code into a function so we can pass its name to run()
def classify_chunks():
    frequent_chunks = [] 
    for start in range(len(dna) - 3): 
        chunk = dna[start:start + 4] 
        if dna.count(chunk) > 50: 
            frequent_chunks.append(chunk) 

    for chunk in frequent_chunks: 
        if chunk in motifs: 
            print(chunk + " is frequent and interesting") 
        else: 
            print(chunk + " is frequent but not interesting")

cProfile.run("classify_chunks()")

AACA is frequent but not interesting
AGCC is frequent but not interesting
TAAC is frequent but not interesting
GGAT is frequent but not interesting
ACCG is frequent but not interesting
TAAC is frequent but not interesting
AACA is frequent but not interesting
GCTT is frequent but not interesting
CAGA is frequent but not interesting
GAAT is frequent but not interesting
ACCG is frequent but not interesting
TAAC is frequent but not interesting
TAAC is frequent but not interesting
AACA is frequent but not interesting
GAAT is frequent but not interesting
GAAT is frequent but not interesting
GCTT is frequent but not interesting
TAAC is frequent but not interesting
TAAC is frequent but not interesting
GCCA is frequent but not interesting
CAGA is frequent but not interesting
TAAC is frequent but not interesting
GCTT is frequent but not interesting
ACCG is frequent but not interesting
AGCC is frequent but not interesting
GCCA is frequent but not interesting
ACCG is frequent but not interesting
G

cProfile gives us tabular output

- ncalls, which tells us how many times the function was called
- tottime, which tells us the total amount of time that was spent in that function (not including sub functions)
- percall, which tells us the amount of time that was spent in that function (not including sub functions) each time it was called
- cumtime, which is like tottime but does include sub functions
- another percall, which is like the first one except that it does include sub functions
- filename, which tells us the filename, line number, and name of the function or method


What can we see from the cProfile output?

- the code took about 0.6 seconds to run (slightly longer than before due to overhead)
- 17877 functions were called

Let's look at some specific functions:

`classify_chunks()`
```
ncalls  tottime  percall  cumtime  percall filename:lineno(function) 
     1  0.009    0.009    0.620    0.620   time_profile.py:13(classify_chunks) 
```
- was called once
- very small tottime i.e. not much going on 
- but very big cumtime, so most of time in functions called inside it

`append()` method of lists
```
ncalls  tottime  percall  cumtime  percall filename:lineno(function) 
   419  0.000    0.000    0.000    0.000   {method 'append' of 'list' objects} 
```
- called 419 times (once for each frequent chunk)
- but tottime and cumtime so low they're rounded to zero

`count()` method of strings
```
ncalls  tottime  percall  cumtime  percall filename:lineno(function) 
  9997  0.612    0.000    0.612    0.000   {method 'count' of 'str' objects} 
```
- called around 10000 times (once per chunk in the DNA sequence)
- percall time is very low - counting is fast
- but tottime is big - most of the time of the program. 

cProfile only measures function calls, i.e. not stuff like

```python
dna[start:start + 4] 
dna.count(chunk) > 50
chunk in motifs
```
To use it well, we need structured code e.g. if we split our code into two functions:

In [28]:
import cProfile 

def get_frequent_chunks(dna): 
    frequent_chunks = [] 
    for start in range(len(dna) - 3): 
        chunk = dna[start:start + 4] 
        if dna.count(chunk) > 50: 
            frequent_chunks.append(chunk) 
    return frequent_chunks 
 
def print_chunks(chunks): 
    for chunk in chunks: 
        if chunk in motifs: 
            print(chunk + " is frequent and interesting") 
        else: 
            print(chunk + " is frequent but not interesting") 
            
def classify_chunks(): 
    frequent_chunks = get_frequent_chunks(dna) 
    print_chunks(frequent_chunks) 

cProfile.run("classify_chunks()") 

GGTG is frequent but not interesting
GTAT is frequent but not interesting
TGGG is frequent and interesting
ACGC is frequent but not interesting
CGCC is frequent and interesting
TGTC is frequent and interesting
ATAC is frequent and interesting
TACG is frequent but not interesting
TGGG is frequent and interesting
GGTG is frequent but not interesting
GTAT is frequent but not interesting
CGCC is frequent and interesting
TCCT is frequent and interesting
CGGT is frequent but not interesting
GGTG is frequent but not interesting
GGAT is frequent but not interesting
GATA is frequent but not interesting
TACG is frequent but not interesting
TTCC is frequent but not interesting
GGAT is frequent but not interesting
GTAT is frequent but not interesting
ATAC is frequent and interesting
TACG is frequent but not interesting
CGGT is frequent but not interesting
TCCG is frequent but not interesting
GATA is frequent but not interesting
ATAC is frequent and interesting
TGTC is frequent and interesting
GTAT

We get more useful output:

```
ncalls tottime percall   cumtime   percall filename:lineno(function) 
 1    0.005    0.005      0.619     0.619  time_profile.py:14(get_frequent_chunks)
 1    0.004    0.004      0.004     0.004  time_profile.py:22(print_chunks) 
```

Most of the time is counting the chunks, not classifying them (hence why our speed-up idea didn't work)

### Profiling with line_profiler

`line_profiler` is a third party module that you have to install separately:

`pip install line_profiler`

It measures execution time per line. To use it we add a `@profile` decorator to the function and run from the command line with:

`kernprof -l -v slowfast.py`

Let's try it:

In [59]:
!kernprof -l -v chunks.py

TCCC is frequent but not interesting
TTCT is frequent but not interesting
TAAG is frequent but not interesting
TAAG is frequent but not interesting
CATC is frequent but not interesting
TAAG is frequent but not interesting
TCCC is frequent but not interesting
CCCA is frequent but not interesting
TAAG is frequent but not interesting
ATCC is frequent and interesting
TCCC is frequent but not interesting
CATC is frequent but not interesting
TCCC is frequent but not interesting
CCCA is frequent but not interesting
CATC is frequent but not interesting
TAAG is frequent but not interesting
ATCC is frequent and interesting
TAAG is frequent but not interesting
TTCT is frequent but not interesting
TTCT is frequent but not interesting
TAAG is frequent but not interesting
CAGG is frequent but not interesting
CAGG is frequent but not interesting
CATC is frequent but not interesting
TAAG is frequent but not interesting
CAGG is frequent but not interesting
TCCC is frequent but not interesting
CAGG is f

Hopefully the output shows the usefulness - we can see immediately that 

`if dna.count(chunk) > 50:`

uses 96% of the time. Let's switch to a dict and keep a tally:

In [ ]:
def classify_chunks(): 
    chunk_count = {}
    for start in range(len(dna) - 3):
        chunk = dna[start:start + 4]
        current_count = chunk_count.get(chunk, 0)
        new_count = current_count + 1 
        chunk_count[chunk] = new_count

    for chunk, count in chunk_count.items():
        if count > 50: 
            if chunk in motifs:
                print(chunk + " is frequent and interesting")
            else:
                print(chunk + " is frequent but not interesting")

In [60]:
!kernprof -l -v chunks.py

GAAG is frequent and interesting
GCTT is frequent and interesting
CCTG is frequent but not interesting
TGGG is frequent but not interesting
TCGT is frequent but not interesting
AAGC is frequent and interesting
AGAA is frequent but not interesting
TTTT is frequent but not interesting
AACA is frequent but not interesting
GGGG is frequent and interesting
CTTT is frequent and interesting
GGCG is frequent and interesting
Wrote profile results to chunks.py.lprof
Timer unit: 1e-06 s

Total time: 0.034628 s
File: chunks.py
Function: classify_chunks at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           @profile
    12                                           def classify_chunks(): 
    13         1            2      2.0      0.0      chunk_count = {}
    14      9998         5982      0.6     17.3      for start in range(len(dna) - 3):
    15      9997         7345      0.7     21.2          chunk = dna[start:start + 4]
    

The improvement is dramatic. We could go on from here to make further improvements. 

### Profiling with realistic inputs

For many programs the data greatly affects the profiling results. Here are functions to calculate AT content and to check if two sequences share the same first five bases:

In [ ]:
def at_content(dna): 
    return (dna.count('A') + dna.count('T')) / len(dna) 

def same_start(dna1, dna2): 
    return dna1[0:5] == dna2[0:5]

We want to take a large collection of DNA sequences and identify those that

- have an AT content greater than some cutoff value and
- have the same first five bases as at least one other sequence

Here's a first attempt:

In [ ]:
def find_interesting(dnas, cutoff):
    interesting = set()
    for one in dnas:
        at = at_content(one)
        if at > cutoff:
            for two in dnas:
                if one != two and same_start(one, two):
                    interesting.add(one)

    return(interesting)

Which function takes the most time? Let's send it though `line_profiler` with 1000 sequences of 1000 bases and a cutoff of 0.54:

In [9]:
!kernprof -v -l realdata.py

Wrote profile results to realdata.py.lprof
Timer unit: 1e-06 s

Total time: 0.007256 s
File: realdata.py
Function: find_interesting at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           @profile
    17                                           def find_interesting(dnas):
    18         1            2      2.0      0.0      interesting = set()
    19      1001          201      0.2      2.8      for one in dnas:
    20      1000         5366      5.4     74.0          at = at_content(one)
    21      1000          232      0.2      3.2          if at > cutoff:
    22      2002          410      0.2      5.7              for two in dnas:
    23      2000         1043      0.5     14.4                  if one != two and same_start(one, two):
    24         1            2      2.0      0.0                      interesting.add(one)
    25                                           
    26         1            0      0.0   

The calls to `at_content()` take most of the time. But, if we try a bigger dataset with 10,000 sequences:

In [11]:
!kernprof -v -l realdata.py

Wrote profile results to realdata.py.lprof
Timer unit: 1e-06 s

Total time: 0.446044 s
File: realdata.py
Function: find_interesting at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           @profile
    17                                           def find_interesting(dnas):
    18         1            2      2.0      0.0      interesting = set()
    19     10001         2091      0.2      0.5      for one in dnas:
    20     10000        54371      5.4     12.2          at = at_content(one)
    21     10000         2385      0.2      0.5          if at > cutoff:
    22    530053       113495      0.2     25.4              for two in dnas:
    23    530000       273465      0.5     61.3                  if one != two and same_start(one, two):
    24       510          234      0.5      0.1                      interesting.add(one)
    25                                           
    26         1            1      1.0   

Now `same_start()` is taking the most time. Why? because it's in the inner loop, so the number of calls scales with the square of the size of the dataset - O(N) vs O(N<sup>2</sup>).

What happens if we keep the dataset size the same at 1000 sequences, but decrease the AT content cutoff so that more sequences pass it (0.525):

In [12]:
!kernprof -v -l realdata.py

Wrote profile results to realdata.py.lprof
Timer unit: 1e-06 s

Total time: 0.046173 s
File: realdata.py
Function: find_interesting at line 16

Line #      Hits         Time  Per Hit   % Time  Line Contents
    16                                           @profile
    17                                           def find_interesting(dnas):
    18         1            2      2.0      0.0      interesting = set()
    19      1001          198      0.2      0.4      for one in dnas:
    20      1000         5412      5.4     11.7          at = at_content(one)
    21      1000          248      0.2      0.5          if at > cutoff:
    22     56056        11726      0.2     25.4              for two in dnas:
    23     56000        28513      0.5     61.8                  if one != two and same_start(one, two):
    24        75           74      1.0      0.2                      interesting.add(one)
    25                                           
    26         1            0      0.0   

`same_start()` dominates again. In both these cases the code is unchanged, but by changing the dataset we get opposite results about which function to optimize first. 

**always carry out benchmarking and profiling on realistic data!**

### Performance optimization tips

- File and network IO is slow, so minimize it
- Existing modules are likely to be fast, so use them (scipy/numpy/pandas)
- Avoid calculating the same thing multiple times
- Functional structures (maps/comprehensions) tend to be faster than loops
- Pick data structures with the properties you want (list vs. set)

and some advanced things to know about:

- write inline code in faster languages
- use parallel code
- let a database do the heavy lifting if you can

## Exercises

**no solutions for these**

See how much faster you can make the `classify_chunks` function from above. What effect do the various parameters (chunk length, DNA sequence length, interesting motif count, frequency cutoff) have on the execution time? what effect do they have on the distribution of execution time across lines?

On the desktop computers, run **Canopy command prompt** then the command line is:

`python "C:\Program Files\Enthought\VE\User\Scripts\kernprof-script.py" -v -l path-to-script`

Do the same with one of your solutions to a previous exercise, or some of your own code. 




In [9]:
# ignore this cell, it's for loading custom js code
from IPython.core.display import Javascript
Javascript(filename="custom.js")

<IPython.core.display.Javascript object>

In [10]:
# ignore this cell, it's for loading custom css code
from IPython.core.display import HTML
HTML(filename="custom.css")